Cross Validation of SARIMAX Forecasting Nuitees dans l'hôtellerie Pays de La Loire PDL 2016 
Set "stepwise = true" in auto-arima. 
Include 3 exogenous variables (meteo,GDP,holidays) in training & test datasets
Predict GDP with auto-arima for 2016 & 2017 and include as exogenous in training&test datasets
Datasets https://github.com/jinnyto/touristcast/tree/datasets/datasets

In [4]:
import pandas as pd
import numpy as np
import datetime 
import matplotlib.pyplot as plt
import pyramid
%matplotlib inline
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
from pyramid.arima import auto_arima
def seasonal_MASE(truth, forecast, seasonality=1):    
    period = truth.shape[0] # T
    # print(period)
    forecast_errors = np.abs(truth - forecast)
    # print(forecast_errors)
    mean_absolute_forecast_error = np.sum(forecast_errors) / period
    # print(mean_absolute_forecast_error)

    naive_period = truth.shape[0] - seasonality # T - m
    # print(naive_period)
    # print(truth[seasonality:])
    # print(truth[:period - seasonality])
    naive_errors = np.abs(truth[seasonality:] - truth[:period - seasonality])
    mean_absolute_naive_error = np.sum(naive_errors) / naive_period
    
    return mean_absolute_forecast_error / mean_absolute_naive_error

In [5]:
dn = pd.read_csv('./datasets/nights_2010-2017.csv',parse_dates=['date'], index_col='date',usecols=[*range(1, 15)])
dm = pd.read_csv('./datasets/avgtemp_2010-2017.csv',parse_dates=['date'],index_col='date',usecols=[*range(1, 15)])
dh = pd.read_csv('./datasets/daysoff_2010-2017.csv',parse_dates=['date'],index_col='date',usecols=['date','daysoff'])
dg = pd.read_csv('./datasets/regionalGDP_2010-2015.csv',parse_dates=['date'],index_col='date',usecols=[*range(1, 15)])
dr = pd.concat([dn, dm,dh,dg], axis=1)
df = dr.dropna()
df = df[['nights_PDL','avgtemp_PDL','gdp_PDL','daysoff']]
exogenous= np.array(df[['avgtemp_PDL','gdp_PDL','daysoff']])
y = np.array(df.nights_PDL)

In [6]:
mc_fit = auto_arima(y=np.array(df.gdp_PDL),start_p=0, start_q=0, max_p=3, max_q=3, m=12,
                    start_P=1, seasonal=False, d=1, D=1, trace=True,
                    error_action='ignore',  # don't want to know if an order does not work
                    suppress_warnings=True,  # don't want convergence warnings
                    stepwise=True)
mc_fit.summary()

Fit ARIMA: order=(0, 1, 0); AIC=1148.679, BIC=1153.204, Fit time=0.026 seconds
Fit ARIMA: order=(1, 1, 0); AIC=1150.440, BIC=1157.228, Fit time=0.116 seconds
Fit ARIMA: order=(0, 1, 1); AIC=1150.404, BIC=1157.192, Fit time=0.025 seconds
Fit ARIMA: order=(1, 1, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Total fit time: 0.191 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                             ARIMA Model Results                              
==============================================================================
Dep. Variable:                    D.y   No. Observations:                   71
Model:                 ARIMA(0, 1, 0)   Log Likelihood                -572.340
Method:                           css   S.D. of innovations            766.766
Date:                Mon, 03 Sep 2018   AIC                           1148.679
Time:                        14:24:16   BIC                           1153.204
Sample:                             1   HQIC                          1150.479
                                                                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        183.9145     90.998      2.021      0.047       5.561     362.268
==============================================================================
"""

In [7]:
mc_fit.fit(df.gdp_PDL)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 0),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=None, solver='lbfgs', start_params=None,
   suppress_warnings=True, transparams=True, trend='c')

In [8]:
next_24 = mc_fit.predict(n_periods=24)
dr.loc['2016-01-01':'2017-12-01','gdp_PDL'] = next_24

In [9]:
mae_fold = []
mse_fold = []
rmse_fold = []
smase_fold= []
startyear = '2010-01-01'
for index in range(2012,2018):
    nextyear = str(index)+'-01-01'
    predictyear = str(index+1)+'-01-01'
    dftrain = dr[(dr.index>=startyear)&(dr.index<nextyear)]
    train_nights  = np.array(dftrain.nights_PDL)
    train_exogenous =  np.array(dftrain[['avgtemp_PDL','gdp_PDL','daysoff']])
    dftest = dr[(dr.index>=nextyear)&(dr.index<predictyear)]
    test_nights = np.array(dftest.nights_PDL)
    test_exogenous =  np.array(dftest[['avgtemp_PDL','gdp_PDL','daysoff']])
    tc_fit = auto_arima(y=train_nights,exogenous=train_exogenous,start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                    start_P=1, seasonal=True, d=1, D=1, trace=True,
                    error_action='ignore',  # don't want to know if an order does not work
                    suppress_warnings=True,  # don't want convergence warnings
                    stepwise=True)
    tc_future_forecast = tc_fit.predict(n_periods=12,exogenous = test_exogenous)
    mae_fold.append(mean_absolute_error(test_nights, tc_future_forecast))
    mse = mean_squared_error(test_nights, tc_future_forecast)
    mse_fold.append(mse)
    rmse_fold.append(sqrt(mse))
    smase_fold.append(seasonal_MASE(test_nights,tc_future_forecast))

print('MAE:')
print(mae_fold)    
print('MSE:')
print(mse_fold)
print('RMSE:')
print(rmse_fold)
print('SMASE:')
print(smase_fold)

Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=117.501, BIC=119.490, Fit time=0.054 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(1, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=117.375, BIC=119.763, Fit time=0.266 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1,

Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=573.443, BIC=587.986, Fit time=0.999 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=552.487, BIC=567.030, Fit time=1.471 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=554.987, BIC=571.608, Fit time=1.744 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=560.782, BIC=573.247, Fit time=0.695 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=555.042, BIC=571.662, Fit time=1.782 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=581.287, BIC=593.752, Fit time=1.042 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 1, 1, 12); AIC=555.548, BIC=572.168, Fit time=1.588 seconds
Fit ARIMA: order=(1, 1, 2) seasonal_order=(0, 1, 1, 12); AIC=557.7